In [2]:
from transformers import pipeline
import torch
import pandas as pd
from tqdm import tqdm
import nltk
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Load the pretrained T5 model and tokenizer
model_name = "NLPnorth/snakmodel-7b-instruct"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:  67%|██████▋   | 2/3 [01:52<00:56, 56.35s/it]


In [3]:
from accelerate import init_empty_weights

model_name = "NLPnorth/snakmodel-7b-instruct"  
tokenizer = AutoTokenizer.from_pretrained(model_name)

# initiating an 'empty' model to use accelerate
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(model_name)

from accelerate import load_checkpoint_and_dispatch

model = load_checkpoint_and_dispatch(
    model, checkpoint='/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/snakmodel-7b-local', device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:2404: UserWarning: for model.embed_tokens.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:2404: UserWarning: for model.layers.0.self_attn.q_proj.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/torch/nn/modules/m

ValueError: At least one of the model submodule will be offloaded to disk, please pass along an `offload_folder`.

In [ ]:
import torch
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
model.to(DEVICE)

# Example question and context
question = "Hvad er retten til barselsdagpenge betinget af?"
context = "§ 3. Retten til barselsdagpenge er betinget af, at personen ved fraværsperiodens begyndelse har lovligt ophold her i landet eller efter EF-forordninger om koordinering af de sociale sikringsordninger, er omfattet af dansk lovgivning om social sikring. Stk. 2. Barselsdagpenge kan dog udbetales under ophold i udlandet til en person, der er udsendt af en arbejdsgiver med hjemsted her i landet til beskæftigelse i højst 1 år i udlandet, når den udsendtes indkomst kan beskattes i Danmark. Stk. 3. Barselsdagpenge kan endvidere udbetales til en person, der ved fraværsperiodens begyndelse opholder sig i udlandet i en periode, som der efter § 27, stk. 3, eller § 28, stk. 2, ses bort fra ved vurderingen af, om beskæftigelseskravet er opfyldt. Stk. 4. Personer, der efter en international aftale om social sikring er omfattet af dansk lovgivning om social sikring, har ret til barselsdagpenge, selv om de ikke har ophold her i landet. Stk. 5. En person, der ved starten af en fraværsperiode er berettiget til barselsdagpenge i overensstemmelse med stk. 1-4, bevarer retten til barselsdagpenge for den resterende del af denne periode, uanset hvor personen opholder sig."

# Format the input for T5
input_text = f"Spørgsmål: {question} Kontekst: {context}"

# Tokenize the input and generate an answer
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(DEVICE)
outputs = model.generate(input_ids, max_length=50)

# Decode and print the generated answer
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Svar:", answer)